***
# TEST
***

In [1]:
import os
import json
import pandas as pd
import pvdeg
from pytest import approx
from pvdeg import TEST_DATA_DIR

#Load weather data
WEATHER = pd.read_csv(os.path.join(TEST_DATA_DIR, 'weather_day_pytest.csv'),
                         index_col= 0, parse_dates=True)
with open(os.path.join(TEST_DATA_DIR, 'meta.json'),'r') as file:
    META = json.load(file)

#Load expected results
rh_expected = pd.read_csv(os.path.join(TEST_DATA_DIR, 'input_day_pytest.csv'),
                          index_col=0, parse_dates=True)
rh_cols = [col for col in rh_expected.columns if 'RH' in col]
rh_expected = rh_expected[rh_cols]


def test_module():
    '''
    test pvdeg.humidity.calc_rel_humidity
    
    Requires:
    ---------
    weather dataframe and meta dictionary
    '''
    result = pvdeg.humidity.module(WEATHER, META)
    pd.testing.assert_frame_equal(result, 
                                  rh_expected, 
                                  check_dtype=False)

def test_psat():
    '''
    test pvdeg.humidity.psat
    
    Requires:
    ---------
    weahter dataframe and meta dictionary
    '''
    psat_avg = pvdeg.humidity.psat(temp=WEATHER['temp_air'])[1]
    assert psat_avg == approx(0.47607, abs=5e-5)



In [3]:
test_psat()

### Symbolic Tests

In [6]:
import os
import json
import pandas as pd
import sympy as sp
import pvdeg
from pvdeg import TEST_DATA_DIR

In [2]:
WEATHER = pd.read_csv(
    os.path.join(TEST_DATA_DIR, r"weather_day_pytest.csv"),
    index_col=0,
    parse_dates=True
)

with open(os.path.join(TEST_DATA_DIR, "meta.json"), "r") as file:
    META = json.load(file)

In [12]:
lnR_0, I, X, Ea, k, T = sp.symbols('lnR_0 I X Ea k T')

ln_R_D_expr = lnR_0 * I**X * sp.exp( (-Ea)/(k * T) )

module_temps = pvdeg.temperature.module(
    weather_df=WEATHER,
    meta=META,
    conf="open_rack_glass_glass"
)

poa_irradiance = pvdeg.spectral.poa_irradiance(
    weather_df=WEATHER, 
    meta=META
    )

module_temps_k = module_temps + 273.15 # convert C -> K

poa_global = poa_irradiance['poa_global'] # take only the global irradiance series from the total irradiance dataframe
poa_global_kw = poa_global / 1000 # [W/m^2] -> [kW/m^2]

values_kwarg = {
    'Ea': 62.08, # activation energy, [kJ/mol]
    'k': 8.31446e-3, # boltzmans constant, [kJ/(mol * K)]
    'T': module_temps_k, # module temperature, [K]
    'I': poa_global_kw, # module plane of array irradiance, [W/m2]
    'X': 0.0341, # irradiance relation, [unitless] 
    'lnR_0': 13.72, # prefactor degradation [ln(%/h)]
}

The array tilt angle was not provided, therefore the latitude tilt of 39.7 was used.
The array azimuth was not provided, therefore an azimuth of 180.0 was used.
The array tilt angle was not provided, therefore the latitude tilt of 39.7 was used.
The array azimuth was not provided, therefore an azimuth of 180.0 was used.


In [14]:
res = pvdeg.symbolic.calc_kwarg_timeseries(
    expr=ln_R_D_expr,
    kwarg=values_kwarg
).sum()

6.561658688478015e-09

In [16]:
import pytest

res == pytest.approx(6.5617e-09)

True

In [ ]:
5 